In [3]:
from app.vectordb import build_database

build_database()

Database built successfully.


Collection(name=naics_collection)

In [1]:
from app.vectordb import load_database

naics_collection = load_database()

/home/athekunal/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/athekunal/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
from app.prediction import Prediction
%load_ext autoreload
%autoreload 2

pred = Prediction(naics_collection)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 42153.81it/s]
/home/athekunal/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
question = "How many software development companies are there in Silicon Valley?"

qemb = pred.pipeline(question)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
from app.gliner import get_entities

entities = get_entities(pred.gliner_model,question)

In [13]:
rdocs = pred.query_database(qemb,"parent")

In [14]:
parent_names = [parent_name[0] for parent_name in rdocs['documents']]

In [25]:
rdocs['embeddings'][0][0]

[0.0613163560628891,
 -0.03473135456442833,
 -0.056598346680402756,
 0.037058401852846146,
 -0.03651108592748642,
 -0.009791308082640171,
 0.08480965346097946,
 -0.04834325611591339,
 -0.0032187849283218384,
 0.004788296762853861,
 0.03770226240158081,
 0.003350866260007024,
 -0.02547856792807579,
 0.025053737685084343,
 0.025323500856757164,
 -0.010605024173855782,
 -0.061377011239528656,
 0.04251238331198692,
 0.019771264865994453,
 -0.11477617174386978,
 -0.06897873431444168,
 -0.11703871190547943,
 -0.026254795491695404,
 -0.002949652262032032,
 -0.1223239079117775,
 -0.006550283171236515,
 0.008629375137388706,
 0.06404194980859756,
 -0.06315355747938156,
 -0.1128331646323204,
 0.004319808445870876,
 0.009722496382892132,
 0.10284310579299927,
 0.025083735585212708,
 -0.03613865375518799,
 0.041247766464948654,
 -0.05731415003538132,
 0.008048772811889648,
 0.00651939120143652,
 -0.029669661074876785,
 0.0037988258991390467,
 0.0025704249273985624,
 -0.03917744383215904,
 -0.08765

In [23]:
child_docs = pred.query_database(rdocs['embeddings'][0][0],"child",parent_names)

In [26]:
parent_names

['Management of Companies and Enterprises']

In [27]:
relevant_docs = pred.naics_collection.query(
            query_embeddings=rdocs['embeddings'][0],
            where={
                "$and": [
                    {"TYPE": {"$eq": "CHILD"}},
                    {"metadata_field": {"$in": parent_names}},
                ]
            },
            n_results=1,
        )

In [28]:
relevant_docs

{'ids': [[]],
 'distances': [[]],
 'metadatas': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'data': None}